## Beginner GeoIPS Tutorial 

Adding Sectors, Feature and Gridline Annotators, Colormappers, and Output Formatters.


Link to these slides can be found [here](https://github.com/NRLMMD-GEOIPS/presentations)

### To get the most out of this tutorial...

[Solutions](https://github.com/NRLMMD-GEOIPS/plugin_tutorial_solutions/tree/main)

Please keep the tutorial solutions up throughout this tutorial

[Beginner Tutorial Part 2 Slides](https://docs.google.com/presentation/d/1tkdRUFjZh_AdA98G1jIjaEjXn_HjYQD3PcweVmTUmTo/edit?slide=id.g2e736e3f9ff_3_94#slide=id.g2e736e3f9ff_3_94)

Also please keep these slides up to copy/paste from here as needed. 


### Tutorial Contents

We're going to show you how to extend GeoIPS by adding new plugins for five new interfaces. 

We will add:  
- One new sector 
- One new feature_annotator , one new gridline_annotator 
- Three new colormappers 
- One new output_formatter

We will also show you how to: 
- Produce a NetCDF4 file using your algorithm output 
- Read your custom NetCDF4 file back in, and produce imagery from it 

### Hands on: Outputting your new product as NetCDF 

1. Copy the Cloud Depth Imagery script to a new file 
    - **Note:** the reason to keep making new scripts rather than editing in place is that 
      you can use all of these later as regression tests for your package! 
2. Edit `clavrx.conus_netcdf.my-cloud-depth-netcdf.sh`

Update `clavrx.conus_netcdf.my-cloud-depth-netcdf.sh` with the following (we updated 
what output_formatter and file_formatter we use):

```bash
#!/bin/bash

geoips run single_source \
   $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
   --reader_name clavrx_hdf4 \
   --product_name My-Cloud-Depth \
   --output_formatter netcdf_geoips \
   --filename_formatter geoips_netcdf_fname \
   --minimum_coverage 0 \
   --sector_list conus
ss_retval=$?

exit $((ss_retval))
```

### Running Your Script 

Activate the code block below to run your modified script.

In [ ]:
%%bash -s "$MY_PKG_DIR"

cd $MY_PKG_DIR/tests/scripts
$MY_PKG_DIR/tests/scripts/clavrx.conus_netcdf.my-cloud-depth.netcdf.sh

This will output a bunch of log output.

If your script succeeded it will end with `INTERACTIVE: Return Value 0`.

To view your output, look for a line that says `SINGLESOURCESUCCESS`.

Open the NetCDF file to view its contents (e.g. with Panoply; shown below)

![My Cloud Depth NetCDF Output](./images/panoply-netcdf-output.png)

**Note** - For more information on the output_formatter 
you just used, run:  

In [ ]:
%%bash

geoips describe output-formatter netcdf_geoips

### A Word About Sectors

Sectors are YAML plugins that define an area of interest for plotting in GeoIPS.

**Two types:** 
- **static**: static sector of a specific area 
- **dynamic**: sectors that are generated at run-time to follow an event (e.g. 
  tropical cyclone, atmospheric river, pyro-Cb, etc.) 

For this tutorial, we will be creating a **static**  sector. 

**Static Sector Properties:** 
- **Metadata**: Contains information related to the origin of the sector. This includes 
  continent, country, state, etc.
- **Projection**: Proj projection information 
  - **a**: The radius of the earth (in meters) 
  - **lat_0**: Center latitude coordinate 
  - **lon_0**: Center longitude coordinate 
  - **proj**: String representing projection type 
  - **units**: String representing units of the 
    projection 
- **Resolution**: The size of each pixel in meters 
- **Shape**: Image shape in pixels (width + height) 
- **Center**: The center x/y coordinates of the sector 

### Example Sectors

![Example Sectors](./images/example_sectors.png)

You can run the following commands to get more information about each of the sectors 
shown here by running:

In [ ]:
%%bash

geoips describe sector australia
geoips describe sector central_america
geoips describe sector south_pole

### Generating Static Sector Properties 

To generate the properties necessary to create a static sector, you must first define 
the area you want to display (captured by the appropriate satellite), based on center 
latitude and longitude bounds. 

The easiest way to do this is to open a [Google Maps](https://www.google.com/maps/) tab, 
then right click the center of your sector to obtain the center latitude and longitude 
values. 

Once you have those center latitude and longitude values, you’re ready to create 
your custom sector. 

The next slide will display the changes you will need to make to 
create a custom conus sector plugin. 

![Google Maps CONUS](./images/google_maps_conus.png)

**Note** - For listing of sector plugins, run: 

In [ ]:
%%bash

geoips list sectors

### Hands on: Creating Custom Sectors 

Let's start by copying a sector file to edit

In [ ]:
%%bash -s "$MY_PKG_DIR" "$MY_PKG_NAME"

# Make a sectors folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static

# Copy a sector file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/yaml/sectors/static/australia.yaml" my_conus_sector.yaml

### Slide 15

Copy sector file to edit 
15Creating a Custom Sector 
Solutions on NRLMMD github.com 
mkdir -pv  $MY_PKG_DIR /$MY_PKG_NAME /plugins/yaml/sectors/static 
cd $MY_PKG_DIR /$MY_PKG_NAME /plugins/yaml/sectors/static 
cp $GEOIPS_PACKAGES_DIR /geoips/geoips/plugins/yaml/sectors/static/australia.yaml my_conus_sector.yaml 
vim my_conus_sector.yaml 

### Slide 16

Generating Static Sector Properties 
The images to the left display the changes you 
will need to make to create a custom conus 
sector plugin. While you can leave the metadata 
untouched, it is very helpful to have additional 
information about the sector being displayed, 
not only for the backend of GeoIPS, but also for 
people using this sector plugin. 
Once you’ve made the appropriate changes, 
you will be ready to use your custom sector 
plugin with CLAVR-x data. Describes the 
projection type 
The resolution 
of each pixel 
in meters 
16Creating a Custom Sector 
Solutions on NRLMMD github.com 
interface: sectors
family: area_definition_static 
name: my_conus_sector 
docstring: "My CONUS Sector" 
metadata: 
  region: 
    continent: NorthAmerica 
    country: UnitedStates 
    area: x
    subarea: x
    state: x
    city: x
spec:
  area_id: my_conus_sector 
  description: CONUS
  projection: 
    a: 6371228.0 
    lat_0: 37.0
    lon_0: -96.0
    proj: eqc
    units: m
  resolution: 
    - 3000
    - 3000
  shape: 
    height: 1000
    width: 2200
  center: [0, 0]interface:  sectors
family: area_definition_static 
name: australia 
docstring: "Australian Continent" 
metadata: 
  region:
    continent: Australia 
    country: x
    area: Continental 
    subarea: x
    state: x
    city: x
spec:
  area_id: australia 
  description: Australian  Continent 
  projection: 
    a: 6371228.0 
    lat_0: -26.5
    lon_0: 134.0
    proj: stere
    units: m
  resolution: 
    - 2000
    - 2000
  shape:
    height: 2100
    width: 2400
  center: [0, 0]Describes the 
projection type 
The resolution 
of each pixel 
in meters 

### Slide 17

Testing Your Custom Static Sector 
The commands you ran in the previous slide create a 
custom conus sector. my_conus_sector.yaml  will be an 
example plugin, showing you that you can create sectors 
just like conus.yaml , to your own specifications. 
To quickly check whether or not you like the shape and 
resolution of your custom sector, you can use the 
command line function create_sector_image . This will 
plot and save images containing the borders and 
coastlines of the inputted sectors. For example, to test 
your custom sector, run the following: 
my_conus_sector.png 
Once completed, open the my_conus_sector.png  
image to see what your sector will look like. 
17Creating a Custom Sector 
Solutions on NRLMMD github.com 
create_plugin_registries 
cd $MY_PKG_DIR /$MY_PKG_NAME 
geoips test sector my_conus_sector 
Note -  You can describe the sector plugin you just 
created by running: 
geoips describe sector my_conus_sector 

### Slide 18

Using Your Custom Static Sector 
To use my_conus_sector.yaml  in your test script, 
simply replace ‘--sector_list conus’  with ‘--sector_list 
my_conus_sector’ . This change means that 
clavrx.conus_annotated.my-cloud-top-height.sh will use 
the sector you just created, rather than the GeoIPS 
conus sector we’ve been using previously. 
# Edit clavrx.my_conus_sector.my-cloud-top-height.sh to use new  my_conus_sector  sector plugin 
18Creating a Custom Sector 
Solutions on NRLMMD github.com 
--sector_list conus --sector_list my_conus_sector cd $MY_PKG_DIR /tests/scripts 
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.my_conus_sector.my-cloud-top-height.sh 
vim clavrx.my_conus_sector.my-cloud-top-height.sh 
$MY_PKG_DIR /tests/scripts/clavrx.my_conus_sector.my-cloud-top-height.sh 


### Slide 19

Hands on: Creating Custom Gridline/Feature 
Annotators 
(feature_annotators  and gridline_annotators 
interfaces) 
19Creating Custom Gridline/Feature Annotators 
Solutions on NRLMMD github.com 

### Slide 20

Defining gridline_annotators and feature_annotators 
gridline_annotators: 
-Describe the format of the grid lines shown 
in your imagery. Can control: 
-Labels , and which to display 
-Lines , such as their color , linestyle , and 
linewidth 
-Spacing , such as the distance between 
latitude  and longitude  labels, and the 
grid lines that represent them. feature_annotators: 
-Describe the format of the features shown 
in your imagery. Can control: 
-Coastlines , Borders , States , and Rivers . 
All of these features have the same 
parameters: 
-Whether they are enabled 
-The color  of the feature 
-The linewidth  of the feature 
displayed 
20

### Slide 21

Example Gridline and Feature 
Annotators 
21Creating Custom Gridline/Feature Annotators 
Solutions on NRLMMD github.com 

### Slide 22

Creating Custom gridline_annotator 
Defines which labels 
will be shown or 
omitted 
You can change these 
values to whatever you 
want! Defines lat/lon spacing 
(can be a float too) Defines line properties. Color 
can be matplotlib named color or 
hex string. Linestyle refers to 
[dash_width, dash_spacing] 
22Creating Custom Gridline/Feature Annotators 
Solutions on NRLMMD github.com 
interface: gridline_annotators 
family: cartopy
name: tutorial 
docstring: |
  The tutorial gridline_annotators configuration.  
  All gridline labels enabled, latitude and 
  Longitude lines colored mediumseagreen, 2.5 degree 
  spacing, 1px linewidth, and [5, 3] linestyle. 
spec:
  labels:  
    top: true
    bottom: true
    left: true
    right: true
  lines:  
    color: mediumseagreen 
    linestyle: [5, 3]
    linewidth: 1
  spacing:  
    latitude: 2.5
    longitude: 2.5mkdir -pv  $MY_PKG_DIR /$MY_PKG_NAME /plugins/yaml/gridline_annotators 
cd $MY_PKG_DIR /$MY_PKG_NAME /plugins/yaml/gridline_annotators 
cp $GEOIPS_PACKAGES_DIR /geoips/geoips/plugins/yaml/gridline_annotators/default.yaml tutorial.yaml 
vim tutorial.yaml 

### Slide 23

Creating Custom feature_annotator 
You can change these 
values to whatever you 
want! Defines if this feature is 
active, it’s color, and it’s 
linewidth 
23Creating Custom Gridline/Feature Annotators 
Solutions on NRLMMD github.com 
interface: feature_annotators 
family: cartopy
name: tutorial 
docstring: |
  The tutorial feature annotators configuration.  
  All line types enabled. All colored  
  [midnightblue, red, yellow, cyan]. 2px coastlines, 
  2px borders, 2px states, 1px rivers .
spec:
  coastline:  
    enabled: true
    edgecolor : midnightblue 
    linewidth : 2
  borders:  
    enabled: true
    edgecolor: red
    linewidth: 2
  states:  
    enabled: true
    edgecolor: darkslategray 
    linewidth: 2
  rivers:  
    enabled: true
    edgecolor: cyan
    linewidth: 1mkdir -pv  $MY_PKG_DIR /$MY_PKG_NAME /plugins/yaml/feature_annotators 
cd $MY_PKG_DIR /$MY_PKG_NAME /plugins/yaml/feature_annotators 
cp $GEOIPS_PACKAGES_DIR /geoips/geoips/plugins/yaml/feature_annotators/default.yaml tutorial.yaml 
vim tutorial.yaml 

### Slide 24

Using Your Custom Annotators 
To implement these changes, simply add the following lines to your test script that uses the 
“imagery_annotated”  output formatter. 
Note -  If you named your custom feature_annotator or gridline_annotators with different names, replace  
‘tutorial’ with such names. Don’t forget you can view your new plugins using geoips describe ! 
Run it: 
 24Creating Custom Gridline/Feature Annotators 
Solutions on NRLMMD github.com 
--feature_annotator tutorial \ 
--gridline_annotator tutorial \ create_plugin_registries 
cd $MY_PKG_DIR /tests/scripts 
cp clavrx.conus_annotated.my-cloud-depth.sh clavrx.conus_annotated_features_gridlines.my-cloud-depth.sh 
vim clavrx.conus_annotated_features_gridlines.my-cloud-depth.sh 
$MY_PKG_DIR /tests/scripts/clavrx.conus_annotated_features_gridlines.my-cloud-depth.sh 

### Slide 25

The Result 
25Creating Custom Gridline/Feature Annotators 
Solutions on NRLMMD github.com 
$MY_PKG_DIR /tests/scripts/clavrx.conus_annotated_features_gridlines.my-cloud-depth.sh 

### Slide 26

Colorbars and Colormaps 
Background and Introduction 
(colormappers  module-based interface) NOTE: This section is informational only - you don’t have to actually change 
anything in your code! 
26

### Slide 27

Using a Matplotlib Colormap 
Matplotlib has many predefined colormaps. 
The matplotlib_linear_norm  colormapper plugin is capable of using 
these predefined colormaps - simply provide the name of the 
colormap in the “cmap_name” argument when you define your 
product plugin. 
For now matplotlib_linear_norm  is the only plugin that can directly 
use named colormaps (which explicitly normalizes the colors using 
matplotlib’s “Normalize” function), though others may be added in 
the future with additional methods of manipulating the color 
scales. 
Sample Product using matplotlib 
built-in colormap 27

### Slide 28

Using an ASCII Colormap 
The matplotlib_linear_norm  plugin can also leverage ASCII colormap files installed within GeoIPS, 
installed within a plugin package, or stored in an arbitrary location on disk. 
To use an ascii colormap, specify  
● cmap_source: ascii 
● cmap_name: <cmap_name>   
Installed ASCII colormaps are stored within a plugin package under plugins/txt/ascii_palettes/ .
● The filename must be of the form <cmap_name>.txt  (this is how geoips finds the given 
cmap_name )
● The colors are defined as RGB triplets ranging from 0 to 256 and should be formatted as three 
columns of 256 integers. 
● Commented lines are allowed (prefaced by “#”), to provide additional context relating to the 
physical meaning of the colormap (ie, min and max expected values/units, transition points, etc) 
● For an example, see the tpw_purple  colormap in the geoips package. 
If you would like t o specify an arbitrary full path on disk rather than installing your ascii palette within your  
plugin package, additionally specify cmap_path . This is most useful for research, development, and testing  
purposes. 
● cmap_path: <full_path>/<any_name>.txt 
matplotlib_linear_norm provides a number of options that can be used to customize how the colormap is  
used and how the colorbar is drawn. 
Sample Product using ascii palette 
based colormap 28

### Slide 29

Using a custom GeoIPS Python-based colormapper 
Color information can also be specified via 
a python-based GeoIPS “colormapper” 
plugin, allowing customization using 
specific matplotlib commands and utilities.  
This is the most flexible method. 
Parameters are optional… but are useful for setting defaults! These? AGAIN?! Optional… but be kind to everyone and provide 
some logging information please. 
(Here’s where gradients are made) Here you set the values 
at which you want the 
colors to transition, as 
well as the colors each 
transition should be 
associated with. 
Gradient creation will 
be handled by GeoIPS. 
Optional. You can import any matplotlib 
manipulations you want! You can define these parameters in a 
number of ways– either in the call function, 
as values here, or even in the 
mpl_colors_info section. Your call, boss. 
29

### Slide 30

Using a custom GeoIPS Python-based colormapper 
The “mpl_colors_info” return dictionary is 
what GeoIPS uses within the 
matplotlib-based utilities and output 
formatters in order to ensure consistent 
application of colors, in both the imagery 
and the colorbars. 
In the next slides, we will walk you through 
creating both GeoIPS Python-based 
colormapper, as well as ASCII-palette 
based matplotlib_linear_norm products. 
Accepts a list of 
alternative (aka 
non-numerical) 
interval labels Scales your values so the color bar 
covers the specified range. Optional. 
30Note -  For a listing of colormappers, run: 
geoips list colormappers 

### Slide 31

Hands on: Creating your own custom 
Python-based colormapper 
31Creating Custom Colormappers 
Solutions on NRLMMD github.com 

### Slide 32

Creating your custom python-based colormapper 
Update the name. 
You can modify the docstring or 
data range if you desire. 32"""Module containing colormap for colorful cloud height product.""" 
import logging 
LOG = logging.getLogger(__name__) 
interface = "colormappers" 
family = "matplotlib" 
name = "colorful_cloud_height" 
def call(data_range=[ 0, 20]):
    """Colorful cloud height colormap. mkdir -pv  $MY_PKG_DIR /$MY_PKG_NAME /plugins/modules/colormappers 
cd $MY_PKG_DIR /$MY_PKG_NAME /plugins/modules/colormappers 
touch __init__.py 
cp $GEOIPS_PACKAGES_DIR /geoips_clavrx/geoips_clavrx/plugins/modules/colormappers/cmap_cldHeight.py colorful_cloud_height.py 
vim colorful_cloud_height.py Creating Custom Colormappers 
Solutions on NRLMMD github.com 

### Slide 33

Creating your custom python-based colormapper 
In this case, I only modified the colors, but you 
can modify anything in this plugin to meet your 
needs.  These are all just direct matplotlib/python 
commands creating the desired color info. 33Creating Custom Colormappers 
Solutions on NRLMMD github.com 
    transition_vals = [ 
        (min_val, 1),
        ( 1, 2),
        ( 2, 3),
        ( 3, 4),
        ( 4, 6),
        ( 6, 8),
        ( 8, 10),
        ( 10, 15),
        ( 15, max_val), 
    ]
    transition_colors = [ 
        ( "pink", "red"),
        ( "paleturquoise" , "teal"),
        ( "plum", "rebeccapurple" ),
        ( "yellow" , "chartreuse" ),
        ( "limegreen" , "darkgreen" ),
        ( "wheat", "darkorange" ),
        ( "darkgray" , "black"),
        ( "lightgray" , "silver" ),
        ( "lightskyblue" , "deepskyblue" ),
    ]


### Slide 34

Hands on: Using your new python-based 
colormapper  in a new product 
34

### Slide 35

Add custom colormapper to new product. 
Create a new Cloud-Base-Python-Colors  product, using 
Cloud-Height as the product_defaults: 
Add a plugin entry to existing my_clavrx_products.yaml  
file.
35Creating Custom Colormappers 
Solutions on NRLMMD github.com 
    - name: Cloud-Base-Python-Colors 
      source_names:  [clavrx] 
      docstring:  |
        CLAVR-x Colorful Cloud Base Height, 
        Using a python-based custom colormapper. 
      product_defaults:  Cloud-Height 
      spec:
        variables:  ["cld_height_base", "latitude", "longitude"] 
        colormapper: 
          plugin: 
            name: colorful_cloud_height 
            arguments: {}cd $MY_PKG_DIR /$MY_PKG_NAME /plugins/yaml/products 
vim my_clavrx_products.yaml 
create_plugin_registries # New plugin entry using colorful_cloud_height  
colormapper plugin 
# Add your new plugins to the registry! 

### Slide 36

Create test script to produce 
your new products 
Run your new product! Edit your new test script! 
36Creating Custom Colormappers 
Solutions on NRLMMD github.com 
cd $MY_PKG_DIR /tests/scripts 
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.conus_annotated.cloud-base-python-colors.sh 
vim clavrx.conus_annotated.cloud-base-python-colors.sh 
$MY_PKG_DIR /tests/scripts/clavrx.conus_annotated.cloud-base-python-colors.sh --product_name My-Cloud-Top-Height 
--product_name Cloud-Base-Python-Colors # Using old product # Using new product 

### Slide 37

Admire the colors 
37


### Slide 38

Hands on: Creating a new Output Formatter 
(output_formatters  python-based interface) 
38

### Slide 39

What are output formatters? 
●Output formatters take native GeoIPS xarray 
data and convert it into something interpretable! 
○NetCDF, an image, HDF5, <you name it!> 
●Required arguments depend on the intent of your 
output! 
Easy! Hard! 
Read all about them in 
$GEOIPS_PKG_DIR/geoips/interfaces/module_based/output_formatters.py 
This one only needs the data, what’s being output and the file name(s)! This one requires all 
the formatting 
information for image 
output! 
39

### Slide 40

Making your own output formatter: 
Step 1: Update this section Step 2: Add some new attributes! 
40Creating Output Formatters 
Solutions on NRLMMD github.com 
mkdir -pv  $MY_PKG_DIR /$MY_PKG_NAME /plugins/modules/output_formatters 
cd $MY_PKG_DIR /$MY_PKG_NAME /plugins/modules/output_formatters 
touch __init__.py 
cp $GEOIPS_PACKAGES_DIR /geoips/geoips/plugins/modules/output_formatters/netcdf_geoips.py my_netcdf_output.py 
vim my_netcdf_output.py Run these! 
interface = "output_formatters" 
family = "xarray_data" 
name = "my_netcdf_output" copy_standard_metadata (xarray_obj, prod_xarray) 
for product_name in product_names: 
    prod_xarray[product_name] = xarray_obj[product_name] 
# Let's add some additional attributes that are necessary for us 
# Add the section below! 
prod_xarray = prod_xarray. assign_attrs (
    Starring= "Richard Karn" ,
    Featuring= "Jonathan Taylor Thomas" ,
    ProducedBy= "Carmen Finestra" ,
)Add this! 
create_plugin_registries Step 3: Register your new product Note -  After you’ve registered your plugin, you can view it using: 
geoips describe output-formatter my_netcdf_output 

### Slide 41

Installing and testing it! 
Inspect the output!  
41Creating Output Formatters 
Solutions on NRLMMD github.com 
cd $MY_PKG_DIR /tests/scripts 
cp clavrx.conus_netcdf.my-cloud-depth-netcdf.sh clavrx.conus_netcdf.my-cloud-top-height-my-netcdf.sh 
vim clavrx.conus_netcdf.my-cloud-top-height-my-netcdf.sh Run these! 
--output_formatter netcdf_geoips --output_formatter my_netcdf_output Update to use your new output_formatter! 
$MY_PKG_DIR /tests/scripts/clavrx.conus_netcdf.my-cloud-top-height-my-netcdf.sh Run your new script! 

### Slide 42

42


### Slide 43

A word about Readers 
(readers  python-based interface) NOTE: This section is informational only - you don’t 
have to actually change anything in your code! 
43

### Slide 44

Readers Overview - Content 
●GeoIPS readers return a dictionary of xarrays minimally containing the 
following variables 
Common Name(s) Required name in xarray Data format 
Latitude, Longitude latitude, longitude float or int
Variables of Interest <customizable but should 
match names used by 
products!> float or int
Time of Observation time datetime object 
Metadata attribute: Data Source source_name string 
Metadata attribute: Time of First 
Observation start_datetime datetime object 
Metadata attribute: Time of Final 
Observation end_datetime datetime object 
44

### Slide 45

Why are those bolded ones so important? 
●Those two determine how GeoIPS interfaces with your reader and product! 
Remember these from your cool new product? This one tells GeoIPS that your source 
is compatible with the product! I.e., if 
your reader’s source_name was 
clavr-x, it wouldn’t work with this spec! 
These are the variable names in the 
product xarray! I.e., if you wrote 
clouds_height_acha  in your product, 
it wouldn’t work! 
45Note -  To get a listing of all reader plugins, run: 
geoips list readers 

### Slide 46

Reader overview - Structure 
●A GeoIPS reader typically consists of the following: 
○The call function  
The call function is the main driver of a GeoIPS reader. It accepts 
the kwargs that contain the list of files to be read, and a handful of 
instructions that adjust how the reader functions. 
○One or several read  functions 
Read  functions populate xarrays with data from the files themselves. Read  functions are technically 
optional… but they are a best practice! 
○Optional utility  functions 
Utility  functions perform some kind of operation on inputs, typically to convert them to a format 
understandable by GeoIPS 
Read function Utility function 
Call function 
These kwargs are important! 
46

### Slide 47

call function key points 
These are the big three pieces of metadata used by 
GeoIPS; source_name  is particularly important as it 
determines which product file is accessed! 
You don’t have to do this… but it’s a really  good idea. You don’t want to read large files twice! 
Remember… readers return a dictionary of xarrays , one of which is the METADATA xarray 
This is a slightly difference approach to the above that reads 
data and then sets the metadata afterward Here is where the read  function is invoked! 
47

### Slide 48

A typical read function 
●Largely a dealer’s choice  (and you’re  the dealer!) 
○The read function needs to open the file and read the contents (remember the minimum 
content slide) into a dictionary of xarrays to be passed along to GeoIPS 
●Common challenges to be aware of: 
○1D variables: It’s okay if your variables are 1D… so long as all of them are 1D . You may need 
to do some array manipulation to get everything even! This is a common issue particularly with 
times! 
○SPeaking of times, time formatting is a common issue: TAI93… UTC… binary string… 
seconds since epoch… there are a lot of ways time is reported in data! Consult the user’s 
guide for your data to figure out what how to convert time variables to the required datetime 
object format. 
○Reading in necessary channels: GeoIPS cannot intelligently read required channels unless 
you code your reader to do so. Remember your call script is invoked with the Chans  
parameter– use that information to save you and your customer’s time! 
48

### Slide 49

An Example Read Function from GMI (1) 
Read in any of the data you need. tb and tb_hi are 
structured a little differently, but more on that later Open the file for reading 
As mentioned earlier, time is a bit finicky! In this 
case, GMI stores each element of the time separately, 
and it only comes as 1D! It needs to be 2D-ified to 
mesh with the 2D lat/long/tb data 
49

### Slide 50

An Example Read Function from GMI (2) 
tb and 
tb_hi are 
all mashed 
together. 
You don’t 
have to 
separate 
them like 
this to fill 
the data 
into an 
xarray, but 
it’s good 
for bean 
counting 
Remember, time has to be 
dtobj 
50

### Slide 51

One last cool thing you can do! 
If your data come as many individual 
granules, and you want users to be able 
to read in many separate files, you can 
use numpy’s vstack function to combine 
the arrays into one larger one! 
Also… notice how this reader didn’t 
reference Chans anywhere? Every time 
this reader is invoked it will read the full 
contents of the file. Maybe not the 
biggest issue for smaller data like GMI… 
but beware for larger datasets! 
51

### Slide 52

Wrap Up 
52

### Slide 53

Additional Information 
●Bonus exercises in the slide package 
○Read that NetCDF4 file to produce imagery 
○Change which colormap you are using to create your imagery 
●GeoIPS Slack Workspace 
https://geoips.slack.com/  
53

### Slide 54

Handy Links 
●Solution repository 
○Github: 
https://github.com/NRLMMD-GEOIPS/plugin_tutorial_solution  
●GeoIPS repository: https://github.com/NRLMMD-GEOIPS/geoips 
54

### Slide 55

Extras 
55

### Slide 56

The Definition of a Plugin 
GeoIPS has two main categories of plugins: 
Module-based 
A python module that defines a new programmatic function 
YAML-based 
A static document that describes how an operation should be performed 
56

### Slide 57

Module Based Plugins 
Module-based Python models containing 
specifically named variables and functions. 
There are a number of different types of 
module-based plugins. 
Today we will create a new algorithm plugin. 
-Algorithms 
-Colormaps 
-Coverage_checkers 
-Filename_formatters 
-Interpolators 
-Output_formatters 
-Procflows 
-Readers 
-Sector_metadata_generators 
-Sector_spec_generators 
-title_formatters 
57

### Slide 58

Module Based Plugins (cont.) 
These plugins largely control the 
functional side of GeoIPS. While 
yaml based plugins define the 
structure of GeoIPS, module based 
plugins control the functional aspect 
of GeoIPS. This includes data flow, 
visualization, managing files, and 
many other aspects as well. To get 
a better understanding of the 
purpose of Module Based Plugins, 
and the overall structure of GeoIPS, 
please refer to this documentation . 
58


### Slide 59

YAML Based Plugins 
YAML based plugins define the structure and usage of the products configurable with 
GeoIPS. When defining a YAML plugin, it’s important to understand how these YAML files 
are structured. Any YAML plugin is defined via it’s “interface”, which refers to a YAML 
schema on GeoIPS that controls the structure of the current “interface” YAML plugin. Here 
are the types of YAML based interfaces that GeoIPS provides: 
-Feature Annotator 
-Gridline Annotator 
-Product defaults 
-Products 
-Dynamic sectors 
-Static sectors 
-ProcFlow configurations 
59

### Slide 60

YAML based Products Plugins 
A products plugin is technically a list of 
products offered by the current 
product_package. For example, if you were 
to create a clavrx.yaml file, all of the 
products listed in this file would be a product 
that is “producible” via clavrx. 
Each product in this list contains their own 
“spec”, which is comprised of variables, and 
other optional arguments which can override 
values initially defined by product defaults. 
(Example Plugin shown to the right.) 
60

### Slide 61

YAML Based product_defaults Plugins 
A product_defaults plugin defines the default state of a product. For example, if you were to 
have a “cloudHeight” product in the aforementioned clavrx.yaml (product plugin) file, you 
could have an associated cloudHeight.yaml product_defaults plugin, which defines the 
default state of a cloudHeight product. 
In the definition of the cloudHeight product in clavrx.yaml, we have the ability to omit such 
variables defined in the cloudHeight.yaml product_defaults plugin, and the ability to override 
such variables set, to our own custom definition. 
This functionality provides flexibility to developers who want to modify products based on 
their own needs. 
61

### Slide 62

Other YAML based plugins 
While the other YAML based plugins are out of the scope of this tutorial, we want to provide 
you with documentation just in case you’d like to dive deeper into these topics. 
To better understand the structure and definitions of other GeoIPS YAML plugins, click here 
and search through the corresponding plugin directories to view their corresponding YAML 
files.
62

### Slide 63

Add custom colormappers to new products. 
Set the name and 
docstring 
appropriately for each 
product! 
Set the colormapper 
plugins appropriately 
for each product! Optional 
cmap_path 
argument for 
cmap_source=
ascii 
63Creating Custom Colormappers 
Solutions on NRLMMD github.com 
Other implementations of colormappers. Specifically, 
those using ascii-based colormaps, and another using 
matplotlib-named colormaps. 

### Slide 64

Add custom colormapper to new product. 
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/products 
vim my_clavrx_products.yaml Create 3 new products, using Cloud-Height as the 
product_defaults: Cloud-Base-ASCII-Palette-Colors , 
Cloud-Base-Python-Colors , and 
Cloud-Base-Builtin-Colors .  
Add to existing my_clavrx_products.yaml  file.
64Creating Custom Colormappers 
Solutions on NRLMMD github.com 

### Slide 65

Create test script to produce 
your new products 
cd $MY_PKG_DIR/tests/scripts 
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.conus_annotated.cloud-base-python-colors.sh 
vim clavrx.conus_annotated.cloud-base-python-colors.sh 
./clavrx.conus_annotated.cloud-base-python-colors.sh 
Run your new product! Edit your new test script! 
65Creating Custom Colormappers 
Solutions on NRLMMD github.com 
geoips run single_source  
$GEOIPS_TESTDATA_DIR /test_data_clavrx/ data/goes16_2023101_1600/ clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207
.level2.hdf \ 
  --reader_name clavrx_hdf4 \ 
  --product_name Cloud-Base-Python-Colors  \
  --output_formatter imagery_annotated \ 
  --filename_formatter geoips_fname \ 
  --minimum_coverage 0 \
  --sector_list conus 
ss_retval=$? 
exit $((ss_retval)) 

### Slide 66

Admire the colors 
66

### Slide 67

Create test scripts to produce your new products 
cd $MY_PKG_DIR/tests/scripts 
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.conus_annotated.cloud-base-ascii-colors.sh 
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.conus_annotated.cloud-base-python-colors.sh 
cp clavrx.conus_annotated.my-cloud-top-height.sh clavrx.conus_annotated.cloud-base-builtin-colors.sh 
./clavrx.conus_annotated.cloud-base-ascii-colors.sh 
./clavrx.conus_annotated.cloud-base-python-colors.sh 
./clavrx.conus_annotated.cloud-base-builtin-colors.sh vim clavrx.conus_annotated.cloud-base-builtin-colors.sh vim clavrx.conus_annotated.cloud-base-ascii-colors.sh 
vim clavrx.conus_annotated.cloud-base-python-colors.sh 
Run your new products! Edit your new test scripts! 
67  --reader_name clavrx_hdf4 \ 
  --product_name Cloud-Base-ASCII-Palette-Colors  \
  --output_formatter imagery_annotated \   --reader_name clavrx_hdf4 \ 
  --product_name Cloud-Base-Builtin-Colors  \
  --output_formatter imagery_annotated \ 
  --reader_name clavrx_hdf4 \ 
  --product_name Cloud-Base-Python-Colors  \
  --output_formatter imagery_annotated \ 

### Slide 68

Tutorial Survey 
●Survey link will be e-mailed out, so please keep an eye out for that - we 
appreciate any and all feedback! 
68